In [1]:
%matplotlib inline
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [81]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import uuid

In [55]:
import tensorflow as tf

In [84]:
# MODEL_ID = str(uuid.uuid4())[:5].upper()
MODEL_ID = "DC274"

In [56]:
LEARNING_RATE = 1e-2
BATCH_SIZE = 32
N_EPOCHS = 25
MOMENTUM = 0.9

In [60]:
((X_train, y_train), (X_test, y_test)) = tf.keras.datasets.fashion_mnist.load_data()
X_train = X_train[:, :, :, np.newaxis]
X_test = X_test[:, :, :, np.newaxis]
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

Model is defined without final softmax layer, so as to have access to logits directly

In [62]:
class MiniVGGNet:
    def build_model(self, width: int, height: int, depth: int, n_classes: int):
        input_shape = (width, height, depth)
        model = tf.keras.models.Sequential(
            [
                tf.keras.layers.Conv2D(
                    32, (3, 3), padding="same", input_shape=input_shape
                ),
                tf.keras.layers.Activation("relu"),
                tf.keras.layers.BatchNormalization(axis=-1),
                tf.keras.layers.Conv2D(32, (3, 3), padding="same"),
                tf.keras.layers.Activation("relu"),
                tf.keras.layers.BatchNormalization(axis=-1),
                tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
                tf.keras.layers.Dropout(0.25),
                # Second block
                tf.keras.layers.Conv2D(32, (3, 3), padding="same"),
                tf.keras.layers.Activation("relu"),
                tf.keras.layers.BatchNormalization(axis=-1),
                tf.keras.layers.Conv2D(32, (3, 3), padding="same"),
                tf.keras.layers.Activation("relu"),
                tf.keras.layers.BatchNormalization(axis=-1),
                tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
                tf.keras.layers.Dropout(0.25),
                # Final block
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(512),
                tf.keras.layers.Activation("relu"),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.Dropout(0.5),
                # Output
                tf.keras.layers.Dense(n_classes),
            ]
        )
        return model

In [63]:
model = MiniVGGNet().build_model(width=28, height=28, depth=1, n_classes=10)
sgd = tf.optimizers.SGD(
    learning_rate=LEARNING_RATE, momentum=MOMENTUM, decay=LEARNING_RATE / N_EPOCHS
)
model.compile(
    optimizer=sgd,
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [64]:
# model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=N_EPOCHS, validation_data=(X_test, y_test))

In [87]:
model.save_weights(f"../models/model_{MODEL_ID}_weights.h5", save_format="h5")

In [85]:
model.save(f"../models/model_{MODEL_ID}")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: ../models/model_DC274/assets


INFO:tensorflow:Assets written to: ../models/model_DC274/assets


In [89]:
model.load_weights("../models/model_DC274_weights.h5")

In [90]:
model.evaluate(X_test, y_test, verbose=2)

313/313 - 3s - loss: 0.1932 - accuracy: 0.9280


[0.19322136044502258, 0.9279999732971191]

Probability model can be created by created a new model and adding a softmax layer at the end:

In [91]:
prob_model = tf.keras.models.Sequential([model, tf.keras.layers.Softmax()])